In [ ]:
!pip install ultralytics
!pip install opencv-python
!pip install ultralytics
!pip install mediapipe
!pip install --upgrade numpy
!pip install --upgrade pandas tensorflow

In [ ]:
import cv2
import numpy as np
import time
from ultralytics import YOLO
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode
from google.colab import files
import mediapipe as mp

# Constants
OBJECT_COLOR = (0, 255, 0)
FACE_COLOR = (0, 0, 255)
HAND_COLOR = (255, 0, 0)
CONF_THRESHOLD = 0.3
KNOWN_DIMENSIONS = {
    "person": 170, "face": 15, "hand": 18, "cell phone": 15, "chair": 45,
    "cup": 8, "bottle": 25, "car": 450, "keyboard": 45,
    "laptop": 35, "mouse": 10, "book": 25, "tv": 100
}
SENSOR_WIDTH = 3.68
IMAGE_WIDTH = 640

# Globals
focalLength = 1000
objectModel = YOLO('yolov8n.pt')
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
mpHands = mp.solutions.hands
handsDetector = mpHands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mpDrawing = mp.solutions.drawing_utils

def calculateDistance(objectClass, bboxWidth, bboxHeight, frameWidth):
    knownDimension = KNOWN_DIMENSIONS.get(objectClass.lower())
    if knownDimension is None:
        bboxSize = max(bboxWidth, bboxHeight)
        distance = (frameWidth * 100) / (bboxSize + 1e-6)
        return distance, False
    if objectClass.lower() in ["face", "hand"]:
        distance = (knownDimension * focalLength) / (bboxWidth + 1e-6)
    else:
        distance = (knownDimension * focalLength) / (max(bboxWidth, bboxHeight) + 1e-6)
    return distance, True

def detectObjects(frame):
    results = objectModel(frame, conf=CONF_THRESHOLD)
    detections = []
    for result in results:
        boxes = result.boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            bboxWidth = x2 - x1
            bboxHeight = y2 - y1
            conf = float(box.conf[0])
            cls = int(box.cls[0])
            className = result.names[cls]
            if className.lower() == "person":
                continue
            distance, isCalculated = calculateDistance(className, bboxWidth, bboxHeight, frame.shape[1])
            detections.append(
            {
                'bbox': (x1, y1, x2, y2),
                'conf': conf,
                'class': className,
                'distance': distance,
                'isCalculated': isCalculated,
                'width': bboxWidth,
                'height': bboxHeight
            })
    return detections

def detectFaces(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    detections = []
    for (x, y, w, h) in faces:
        distance, isCalculated = calculateDistance("face", w, h, frame.shape[1])
        detections.append(
        {
            'bbox': (x, y, x + w, y + h),
            'distance': distance,
            'isCalculated': isCalculated,
            'width': w,
            'height': h,
            'class': 'face'
        })
    return detections

def detectHands(frame):
    rgbFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = handsDetector.process(rgbFrame)
    detections = []
    if results.multi_hand_landmarks:
        for handLandmarks in results.multi_hand_landmarks:
            landmarksArray = np.array([(lm.x * frame.shape[1], lm.y * frame.shape[0]) for lm in handLandmarks.landmark])
            xMin, yMin = landmarksArray.min(axis=0).astype(int)
            xMax, yMax = landmarksArray.max(axis=0).astype(int)
            bboxWidth = xMax - xMin
            bboxHeight = yMax - yMin
            distance, isCalculated = calculateDistance("hand", bboxWidth, bboxHeight, frame.shape[1])
            detections.append(
            {
                'bbox': (xMin, yMin, xMax, yMax),
                'distance': distance,
                'isCalculated': isCalculated,
                'width': bboxWidth,
                'height': bboxHeight,
                'class': 'hand',
                'landmarks': handLandmarks
            })
    return detections

def processFrame(frame):
    resultFrame = frame.copy()
    objects = detectObjects(frame)
    faces = detectFaces(frame)
    hands = detectHands(frame)

    for obj in objects:
        x1, y1, x2, y2 = obj['bbox']
        cv2.rectangle(resultFrame, (x1, y1), (x2, y2), OBJECT_COLOR, 2)
        distanceText = f"{obj['distance']:.1f}cm" if obj['isCalculated'] else f"~{obj['distance']:.1f}cm"
        label = f"{obj['class']}{obj['conf']:.2f}:{distanceText}"
        cv2.putText(resultFrame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, OBJECT_COLOR, 2)

    for face in faces:
        x1, y1, x2, y2 = face['bbox']
        cv2.rectangle(resultFrame, (x1, y1), (x2, y2), FACE_COLOR, 2)
        distanceText = f"{face['distance']:.1f}cm" if face['isCalculated'] else f"~{face['distance']:.1f}cm"
        label = f"Face: {distanceText}"
        cv2.putText(resultFrame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, FACE_COLOR, 2)

    for hand in hands:
        x1, y1, x2, y2 = hand['bbox']
        cv2.rectangle(resultFrame, (x1, y1), (x2, y2), HAND_COLOR, 2)
        distanceText = f"{hand['distance']:.1f}cm" if hand['isCalculated'] else f"~{hand['distance']:.1f}cm"
        label = f"Hand:{distanceText}"
        cv2.putText(resultFrame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, HAND_COLOR, 2)
        if 'landmarks' in hand:
            mpDrawing.draw_landmarks(resultFrame, hand['landmarks'], mpHands.HAND_CONNECTIONS,
                                     mpDrawing.DrawingSpec(color=HAND_COLOR, thickness=2, circle_radius=2),
                                     mpDrawing.DrawingSpec(color=HAND_COLOR, thickness=2, circle_radius=2))
    return resultFrame

def calibrateCamera(knownDistance, knownWidth, frameWidth):
    global focalLength
    focalLength = (knownWidth * frameWidth) / (knownDistance + 1e-6)
    print(f"Calibrated focal length: {focalLength:.2f} pixels")

def cv2Imshow(image):
    _, encodedImage = cv2.imencode('.jpg', image)
    display(Image(data=encodedImage.tobytes()))

def runDetectorOnWebcam():
    js = """<your JavaScript code here>"""  # Skipped for brevity

    try:
        dummyFrame = np.zeros((480, 640, 3), dtype=np.uint8)
        processFrame(dummyFrame)
        print("Models loaded successfully!")
        print("Starting webcam capture. Press Ctrl+C to stop.")

        while True:
            display(Javascript(js))
            result = eval_js("captureWebcam()")
            if not result['success']:
                print(f"Error accessing webcam: {result.get('error', 'Unknown error')}")
                break
            imageData = b64decode(result['image'])
            imageArray = np.frombuffer(imageData, dtype=np.uint8)
            frame = cv2.imdecode(imageArray, cv2.IMREAD_COLOR)
            resultFrame = processFrame(frame)
            cv2Imshow(resultFrame)
            time.sleep(0.5)
    except KeyboardInterrupt:
        print("\nWebcam capture stopped")
    except Exception as e:
        print(f"An error occurred: {e}")

def runDetectorOnImage(imagePath):
    frame = cv2.imread(imagePath)
    if frame is None:
        print(f"Error: Could not read image from {imagePath}")
        return
    resultFrame = processFrame(frame)
    print(f"Detection results for {imagePath}:")
    cv2Imshow(resultFrame)

def testWithSampleImage():
    print("Please upload an image file:")
    uploaded = files.upload()
    if uploaded:
        imagePath = list(uploaded.keys())[0]
        runDetectorOnImage(imagePath)
    else:
        print("No image uploaded.")

def main():
    print("Enhanced Object, Face and Hand Detection System with Distance Estimation")
    print("----------------------------------------------------------------------")
    print("1. Run with webcam (requires Colab)")
    print("2. Test with a sample image")
    print("3. Exit")

    while True:
        choice = input("\nEnter your choice (1-3): ")
        if choice == '1':
            try:
                import ultralytics
                import mediapipe
            except ImportError:
                print("Installing required packages...")
                !pip install ultralytics opencv-python-headless mediapipe
            runDetectorOnWebcam()
            break
        elif choice == '2':
            testWithSampleImage()
            break
        elif choice == '3':
            print("Exiting...")
            break
        else:
            print("Invalid choice! Please enter 1, 2, or 3.")

if __name__ == "__main__":
    main()
